In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:



df = pd.read_csv('injury-details-2025-q3.csv')


london_boroughs = [
    'Barking & Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
    'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich',
    'Hackney', 'Hammersmith & Fulham', 'Haringey', 'Harrow',
    'Havering', 'Hillingdon', 'Hounslow', 'Islington',
    'Kensington & Chelsea', 'Kingston Upon Thames', 'Lambeth',
    'Lewisham', 'Merton', 'Newham', 'Redbridge',
    'Richmond Upon Thames', 'Southwark', 'Sutton',
    'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster',
    'City Of London'
]


df_london = df[df['Borough'].isin(london_boroughs)].copy()



borough_counts = df_london['Borough'].value_counts().reset_index()
borough_counts.columns = ['Borough', 'number_of_incidents']


severe_mask = df_london['Injury outcome'].astype(str).str.contains('taken to hospital', case=False, na=False)
severe_counts = df_london[severe_mask]['Borough'].value_counts().reset_index()
severe_counts.columns = ['Borough', 'number_of_severe_incidents']


result = pd.merge(borough_counts, severe_counts, on='Borough', how='left')
result['number_of_severe_incidents'] = result['number_of_severe_incidents'].fillna(0).astype(int)
result = result.sort_values('number_of_incidents', ascending=False).reset_index(drop=True)
result.to_csv('CSVForQuestion1.csv', index=False)





In [5]:
import geopandas as gpd
import pandas as pd

gdf = gpd.read_file("LondonGeo.json", driver='TopoJSON', layer='boroughs')

gdf['longitude'] = gdf.centroid.x
gdf['latitude'] = gdf.centroid.y

centroids = gdf[['id', 'longitude', 'latitude']].copy()
centroids.columns = ['Borough', 'longitude', 'latitude']

incidents = pd.read_csv("CSVForQuestion1.csv")
merged = incidents.merge(centroids, on='Borough', how='left')

# Make absolutely sure no index is saved
merged.to_csv("Centroids.csv", index=False)  # ✓ You had this

# But verify the output
with open('Centroids.csv', 'r') as f:
    print("First line:", f.readline())


First line: Borough,number_of_incidents,number_of_severe_incidents,longitude,latitude



c:\Users\mason\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: driver TopoJSON does not support open option DRIVER
  return ogr_read(


In [4]:
import os

# Check file size
print(f"File size: {os.path.getsize('Centroids.csv') / 1024:.2f} KB")

# Check the actual content
print(merged.head())
print(merged.dtypes)
print(merged.info())

# Check for any weird data in the float columns
print(merged['longitude'].describe())
print(merged['latitude'].describe())

# Look at the actual CSV content
with open('Centroids.csv', 'r') as f:
    lines = f.readlines()
    print(f"Number of lines: {len(lines)}")
    print(f"First line length: {len(lines[0])}")
    print("First 3 lines:")
    for line in lines[:3]:
        print(repr(line))  # repr will show hidden characters

File size: 2.08 KB
       Borough  number_of_incidents  number_of_severe_incidents  longitude  \
0  Westminster                 4253                         916  -0.161088   
1      Lambeth                 4189                         782  -0.118211   
2    Southwark                 3871                         780  -0.074563   
3      Croydon                 3642                         699  -0.087150   
4     Lewisham                 3110                         562  -0.020113   

    latitude  
0  51.513789  
1  51.453274  
2  51.473117  
3  51.355403  
4  51.448144  
Borough                        object
number_of_incidents             int64
number_of_severe_incidents      int64
longitude                     float64
latitude                      float64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0